In [ ]:
import pandas as pd
import numpy as np
import torch
import warnings
import sys, os
import torch
from torch.nn import nn
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

sys.path.append(os.path.abspath('..'))

ais_type_label_path = '../../data/labels/ais_type_labels_radar_detections.csv'
image_path = 'track_images/'

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

import warnings
warnings.filterwarnings("ignore", message="Failed to load image Python extension")

Using device: mps


#### Background

Run track_rasterizer.py: converts cleaned_radar_detections into images stored in track_images, ready to use

In [64]:
BATCH_SIZE = 32
NUM_HIDDEN_1 = 32
NUM_HIDDEN_2 = 64
NUM_HIDDEN_3 = 128

from core.DICT import TYPE_to_LABEL
NUM_CLASSES = len(TYPE_to_LABEL)

In [33]:
from dataloader_imgs import get_type_datasets

train_loader, val_loader, test_loader = get_type_datasets(ais_type_label_path, image_path, batch_size = BATCH_SIZE)

In [38]:
for next_batch in train_loader:
    print(next_batch[0].shape)
    print(next_batch[1].shape)
    print(next_batch[1])
    break

/Users/liuzehan/miniconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/liuzehan/miniconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <2BD1B165-EC09-3F68-BCE4-8FE4E70CA7E2> /Users/liuzehan/miniconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <25C510F7-7AEE-3D64-80ED-95874DC6BECD> /Users/liuzehan/miniconda3/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


torch.Size([32, 3, 224, 224])
torch.Size([32])
tensor([2, 0, 1, 1, 1, 0, 1, 2, 2, 0, 1, 2, 1, 1, 6, 6, 1, 2, 0, 2, 1, 1, 3, 1,
        6, 6, 2, 1, 2, 6, 1, 2])


In [45]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNBaseline(nn.Module):
    def __init__(self,  num_hidden_1, num_hidden_2, num_hidden_3, num_classes):
        super(CNNBaseline, self).__init__()
        self.num_hidden_1 = num_hidden_1
        self.num_hidden_2 = num_hidden_2
        self.num_hidden_3 = num_hidden_3
        self.num_classes = num_classes
        
        self.features = nn.Sequential(
            # Conv Block 1
            # Input: 3*224*224
            nn.Conv2d(in_channels=3, out_channels=self.num_hidden_1, kernel_size=3, padding=1),  # -> (h1, 224, 224)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # -> (h1, 112, 112)
            nn.Dropout(p=0.25),

            # Conv Block 2
            nn.Conv2d(self.num_hidden_1, self.num_hidden_2, kernel_size=3, padding=1),  # -> (h2, 112, 112)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # -> (h2, 56, 56)
            nn.Dropout(p=0.25),

            # Conv Block 3
            nn.Conv2d(self.num_hidden_2, self.num_hidden_3, kernel_size=3, padding=1),  # -> (h3, 56, 56)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # -> (h3, 28, 28)
            nn.Dropout(p=0.25)
        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),  # -> (h3 * 28 * 28)
            nn.Linear(self.num_hidden_3 * 28 * 28, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(512, self.num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [72]:
from torch.utils.data import DataLoader
import copy

def train(model: nn.Module, 
                train_loader: DataLoader, 
                val_loader: DataLoader, 
                optimizer: optim.Optimizer, 
                device: torch.device,
                num_epochs: int,
                scheduler: None, ) -> tuple[list[float], list[float], list[float], list[float]]:
    
    TRAIN_LOSSES = []
    TRAIN_ACC = []
    VAL_LOSSES = []
    VAL_ACC = []

    best_val_loss = float('inf')
    best_model_wts = copy.deepcopy(model.state_dict())

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, targets in tqdm(train_loader, desc="Training", leave=False):
            inputs, targets = inputs.to(device), targets.to(device) 
            optimizer.zero_grad()
            outputs = model(inputs)
            criterion = nn.CrossEntropyLoss()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()    

            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            correct += (predicted == targets).sum().item()
            total += targets.size(0)
            
        train_loss = running_loss / total
        train_acc = correct / total
        TRAIN_LOSSES.append(train_loss)
        TRAIN_ACC.append(train_acc)

        if scheduler is not None:
            scheduler.step()

        model.eval()
        running_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():   
            for inputs, targets in tqdm(val_loader, desc="Evaluating", leave=False):
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                running_loss += loss.item() * inputs.size(0)
                _, predicted = outputs.max(1)
                correct += (predicted == targets).sum().item()
                total += targets.size(0)

        val_loss = running_loss / total
        val_acc = correct / total
        VAL_LOSSES.append(val_loss)
        VAL_ACC.append(val_acc) 

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model_wts = copy.deepcopy(model.state_dict())

        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
        print(f"Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

    return TRAIN_LOSSES, TRAIN_ACC, VAL_LOSSES, VAL_ACC, best_model_wts, best_val_loss



In [ ]:
cnn_baseline_1 = CNNBaseline(NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_HIDDEN_3, NUM_CLASSES).to(device)

train_config = {
    "model": cnn_baseline_1,
    "train_loader" : train_loader,
    "val_loader" : val_loader,
    "optimizer" : optim.Adam(cnn_baseline_1.parameters(), lr=1e-3),
    "device" : device,
    "num_epochs" : 10,
    "scheduler": None
}

train_loss, train_acc, val_loss, val_acc, best_model_wts, best_val_loss = train(**train_config)
torch.save(best_model_wts, "models/cnn_baseline_best_10.pth")




Epoch 1/10
Train Loss: 1.1232 | Train Acc: 0.6322
Val   Loss: 0.9439 | Val   Acc: 0.6852


Epoch 2/10
Train Loss: 0.9298 | Train Acc: 0.6904
Val   Loss: 0.9134 | Val   Acc: 0.6915


Epoch 3/10
Train Loss: 0.8850 | Train Acc: 0.7047
Val   Loss: 0.9035 | Val   Acc: 0.6859


Epoch 4/10
Train Loss: 0.8403 | Train Acc: 0.7112
Val   Loss: 0.8575 | Val   Acc: 0.7092


Epoch 5/10
Train Loss: 0.8015 | Train Acc: 0.7278
Val   Loss: 0.8583 | Val   Acc: 0.7106


Epoch 6/10
Train Loss: 0.7646 | Train Acc: 0.7402
Val   Loss: 0.8512 | Val   Acc: 0.7120


Epoch 7/10
Train Loss: 0.7224 | Train Acc: 0.7511
Val   Loss: 0.8579 | Val   Acc: 0.7063


Epoch 8/10
Train Loss: 0.6770 | Train Acc: 0.7699
Val   Loss: 0.8543 | Val   Acc: 0.7211


Epoch 9/10
Train Loss: 0.6361 | Train Acc: 0.7848
Val   Loss: 0.8805 | Val   Acc: 0.7077


Epoch 10/10
Train Loss: 0.5966 | Train Acc: 0.7983
Val   Loss: 0.8858 | Val   Acc: 0.7085


In [80]:
# 1. Rebuild the model architecture
cnn_baseline_1 = CNNBaseline(NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_HIDDEN_3, NUM_CLASSES).to(device)

# 2. Load the previously saved weights
checkpoint_path = "models/cnn_baseline_best_10.pth"
cnn_baseline_1.load_state_dict(torch.load(checkpoint_path))

# 3. Set up the optimizer again
optimizer = optim.Adam(cnn_baseline_1.parameters(), lr=1e-4)

# 4. Define the new training config (next 10 epochs)
train_config_resume = {
    "model": cnn_baseline_1,
    "train_loader": train_loader,
    "val_loader": val_loader,
    "optimizer": optimizer,
    "device": device,
    "num_epochs": 10,
    "scheduler": None
}

# 5. Resume training
train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2 = train(**train_config_resume)

# 6. Save the updated best model
torch.save(best_model_wts_2, "models/cnn_baseline_best_20.pth")


Epoch 1/10
Train Loss: 0.6839 | Train Acc: 0.7646
Val   Loss: 0.8251 | Val   Acc: 0.7120


Epoch 2/10
Train Loss: 0.6641 | Train Acc: 0.7706
Val   Loss: 0.8265 | Val   Acc: 0.7169


Epoch 3/10
Train Loss: 0.6527 | Train Acc: 0.7759
Val   Loss: 0.8300 | Val   Acc: 0.7155


Epoch 4/10
Train Loss: 0.6433 | Train Acc: 0.7833
Val   Loss: 0.8349 | Val   Acc: 0.7148


Epoch 5/10
Train Loss: 0.6305 | Train Acc: 0.7817
Val   Loss: 0.8395 | Val   Acc: 0.7162


Epoch 6/10
Train Loss: 0.6143 | Train Acc: 0.7879
Val   Loss: 0.8421 | Val   Acc: 0.7197


Epoch 7/10
Train Loss: 0.6079 | Train Acc: 0.7916
Val   Loss: 0.8451 | Val   Acc: 0.7183


Epoch 8/10
Train Loss: 0.5869 | Train Acc: 0.8011
Val   Loss: 0.8503 | Val   Acc: 0.7225


Epoch 9/10
Train Loss: 0.5805 | Train Acc: 0.8043
Val   Loss: 0.8563 | Val   Acc: 0.7190


Epoch 10/10
Train Loss: 0.5671 | Train Acc: 0.8076
Val   Loss: 0.8562 | Val   Acc: 0.7197


In [81]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNNSimple(nn.Module):
    def __init__(self,  num_hidden_1, num_hidden_2, num_classes):
        super(CNNSimple, self).__init__()
        self.num_hidden_1 = num_hidden_1
        self.num_hidden_2 = num_hidden_2
        self.num_classes = num_classes
        
        self.features = nn.Sequential(
            # Conv Block 1
            # Input: 3*224*224
            nn.Conv2d(in_channels=3, out_channels=self.num_hidden_1, kernel_size=3, padding=1),  # -> (h1, 224, 224)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # -> (h1, 112, 112)
            nn.Dropout(p=0.25),

            # Conv Block 2
            nn.Conv2d(self.num_hidden_1, self.num_hidden_2, kernel_size=3, padding=1),  # -> (h2, 112, 112)
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),  # -> (h2, 56, 56)
            nn.Dropout(p=0.25),

        )
        
        self.classifier = nn.Sequential(
            nn.Flatten(),  # -> (h3 * 56 * 56)
            nn.Linear(self.num_hidden_2 * 56 * 56, 256),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(256, self.num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [82]:
NUM_HIDDEN_1 = 16
NUM_HIDDEN_2 = 32

cnn_simple = CNNSimple(NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_CLASSES).to(device)

train_config = {
    "model": cnn_simple,
    "train_loader" : train_loader,
    "val_loader" : val_loader,
    "optimizer" : optim.Adam(cnn_simple.parameters(), lr=1e-3),
    "device" : device,
    "num_epochs" : 10,
    "scheduler": None
}

train_loss, train_acc, val_loss, val_acc, best_model_wts, best_val_loss = train(**train_config)
torch.save(best_model_wts, "models/cnn_simple_best_10.pth")

Epoch 1/10
Train Loss: 1.1389 | Train Acc: 0.6430
Val   Loss: 1.0041 | Val   Acc: 0.6585


Epoch 2/10
Train Loss: 0.9428 | Train Acc: 0.6862
Val   Loss: 0.9938 | Val   Acc: 0.6585


Epoch 3/10
Train Loss: 0.9008 | Train Acc: 0.7011
Val   Loss: 0.9862 | Val   Acc: 0.6556


Epoch 4/10
Train Loss: 0.8569 | Train Acc: 0.7072
Val   Loss: 0.9316 | Val   Acc: 0.6810


Epoch 5/10
Train Loss: 0.8173 | Train Acc: 0.7240
Val   Loss: 0.9165 | Val   Acc: 0.6796


Epoch 6/10
Train Loss: 0.7721 | Train Acc: 0.7403
Val   Loss: 0.9195 | Val   Acc: 0.6880


Epoch 7/10
Train Loss: 0.7271 | Train Acc: 0.7523
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 8/10
Train Loss: 0.6780 | Train Acc: 0.7672
Val   Loss: 0.9346 | Val   Acc: 0.7007


Epoch 9/10
Train Loss: 0.6216 | Train Acc: 0.7830
Val   Loss: 0.9535 | Val   Acc: 0.6915


Epoch 10/10
Train Loss: 0.5749 | Train Acc: 0.8082
Val   Loss: 0.9875 | Val   Acc: 0.6958


In [83]:
# 1. Rebuild the model architecture
cnn_simple = CNNSimple(NUM_HIDDEN_1, NUM_HIDDEN_2, NUM_CLASSES).to(device)

# 2. Load the previously saved weights
checkpoint_path = "models/cnn_simple_best_10.pth"
cnn_simple.load_state_dict(torch.load(checkpoint_path))

# 3. Set up the optimizer again
optimizer = optim.Adam(cnn_baseline_1.parameters(), lr=1e-4)

# 4. Define the new training config (next 10 epochs)
train_config_resume = {
    "model": cnn_simple,
    "train_loader": train_loader,
    "val_loader": val_loader,
    "optimizer": optimizer,
    "device": device,
    "num_epochs": 10,
    "scheduler": None
}

# 5. Resume training
train_loss_2, train_acc_2, val_loss_2, val_acc_2, best_model_wts_2, best_val_loss_2 = train(**train_config_resume)

# 6. Save the updated best model
torch.save(best_model_wts_2, "models/cnn_simple_best_20.pth")


Epoch 1/10
Train Loss: 0.7297 | Train Acc: 0.7483
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 2/10
Train Loss: 0.7328 | Train Acc: 0.7471
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 3/10
Train Loss: 0.7310 | Train Acc: 0.7482
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 4/10
Train Loss: 0.7335 | Train Acc: 0.7506
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 5/10
Train Loss: 0.7370 | Train Acc: 0.7487
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 6/10
Train Loss: 0.7396 | Train Acc: 0.7477
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 7/10
Train Loss: 0.7268 | Train Acc: 0.7444
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 8/10
Train Loss: 0.7324 | Train Acc: 0.7449
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 9/10
Train Loss: 0.7283 | Train Acc: 0.7482
Val   Loss: 0.9145 | Val   Acc: 0.6993


Epoch 10/10
Train Loss: 0.7365 | Train Acc: 0.7426
Val   Loss: 0.9145 | Val   Acc: 0.6993
